<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna
!pip install LAMDA-SSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection
from statistics import mean

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

Evalueringsmetrikker

In [4]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

# Algoritme: SSL Regression Heat Diffusion on graphs

In [5]:
from scipy import sparse
#from gbssl import LGC,HMN,PARW,OMNI,CAMLP
import random
import numpy as np
from sklearn.neighbors import kneighbors_graph

from sklearn.preprocessing import normalize
from scipy.io import loadmat

from sklearn.metrics.pairwise import rbf_kernel
from scipy.sparse import lil_matrix
from scipy import linalg
from sklearn.metrics import log_loss

from sklearn.neighbors import KNeighborsRegressor
from scipy.sparse.linalg import spsolve
from scipy.sparse import identity
import math
import warnings
warnings.filterwarnings("ignore")

In [6]:
def get_graph_matrix(X, k):

    # KNN graph
    KNN_graf = kneighbors_graph(X,
                                n_neighbors=k,
                                mode ="connectivity",
                                include_self=False)

    # Sigma
    n_egenskaper = X.shape[1]
    sigma = 1.0/ n_egenskaper

    # Normalisering for å unnga bias
    #distanser = np.sqrt((KNN_graf ** 2).sum(axis=1))
    #normalisert_KNN_graf = KNN_graf / distanser[: , np.newaxis]

    # RBF funksjon for vekter
    RBF_vekter = rbf_kernel(X, X, gamma=1.0 / (2 * sigma ** 2))

    KNN_RBF_graf = sparse.csr_matrix.multiply(KNN_graf, RBF_vekter)

    return KNN_RBF_graf

def get_harmonic_scores(G,treningsnode_markert_id, treningsnode_umarkert_id,testnode_id,yl):

    ##create Propagation Matrix###

    n = G.shape[0]
    graph = G.copy()
    degrees = graph.sum(axis=0).A[0]
    degrees[degrees==0] += 1  # Avoid division by 0
    D = sparse.diags((1.0/degrees),offsets=0)
    P = D.dot(graph).tolil() # Transition matrix
    P[treningsnode_markert_id] = 0
    all_values = np.zeros(n)
    all_values[treningsnode_markert_id] = 1

    all_values[treningsnode_umarkert_id] = 0
    all_values[testnode_id] = 0
    state_vector = all_values.copy()


    ###Propagate the scores####
    remaining_iter = 30
    state = state_vector.copy()
    Base = state.copy() # Initialize U = Y
    P = P.A

    while remaining_iter > 0:
        state = P.dot(state) + Base # Y^(k+1) <- Y^(k+1) + U
        remaining_iter -= 1

    state = np.round_(state, decimals=2)

    return state


def bhd(X_data, y_data, treningsnoder_markert, treningsnoder_umarkert, testnoder, k, alpha=1.0, maks_iterasjoner=100):

  graph_data = get_graph_matrix(X_data, k)
  y_trening_markert = y_data[treningsnoder_markert]

  state_hmn = get_harmonic_scores(graph_data,treningsnoder_markert, treningsnoder_umarkert, testnoder, y_trening_markert)

  graph = graph_data.copy()
  n = graph.shape[0]
  all_values = y_data.copy() # label vector
  all_values[testnoder] = np.mean(all_values[treningsnoder_markert]) #We imputed the values for the inlabeled nodes as the means of the labeled nodes
  all_values[treningsnoder_umarkert] = np.mean(all_values[treningsnoder_markert])
  remaining_iter = maks_iterasjoner

  I = np.eye(n,n,dtype=np.float64) # Identity matrix
  C = all_values - state_hmn # Constant C
  state = C.copy() # state vector
  state = state.reshape(n,1)
  L = sparse.csgraph.laplacian(graph,normed=True) # Laplacian matrix
  V = I + (-alpha/remaining_iter) * L

  while remaining_iter > 0:
      state = V.dot(state)
      remaining_iter -= 1

  state = state_hmn.reshape(n,1) + state

  y_prediksjon = state

  y_trening_prediksjon = np.ravel(y_prediksjon[treningsnoder_markert])
  y_test_prediksjon = np.ravel(y_prediksjon[testnoder])

  return y_test_prediksjon, y_trening_prediksjon

# Egendefinerte moduler

In [7]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [8]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import umarkert_andel_dag
from oppdeling_trening_testsett import enzymfordeling

In [9]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_semiveiledet.py /content/moduler_optimalisering_evaluering_semiveiledet.py

/content


In [10]:
from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_enzym

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_dag

# Importering av relevant data

In [11]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
collagen_data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

collagen_data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

collagen_data_normal.index = pd.to_datetime(collagen_data_normal.index,
                                            format='%Y-%m-%d %H:%M:%S')

collagen_data_design.index = pd.to_datetime(collagen_data_design.index,
                                            format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [12]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [13]:
collagen_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29049 entries, 2022-10-31 17:37:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29049 non-null  int64  
 1   EnzymeType_A2  29049 non-null  int64  
 2   EnzymeType_B   29049 non-null  int64  
 3   EnzymeType_C   29049 non-null  int64  
 4   EnzymeType_D   29049 non-null  int64  
 5   EnzymeType_E   29049 non-null  int64  
 6   RawMatPercent  29049 non-null  float64
 7   NIRfat         29049 non-null  float64
 8   NIRash         29049 non-null  float64
 9   NIRwater       29049 non-null  float64
 10  TT08           29049 non-null  float64
 11  TT20           29049 non-null  float64
 12  TT12           29049 non-null  float64
 13  Collagen       89 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [14]:
collagen_data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18542 entries, 2022-11-02 00:00:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18542 non-null  int64  
 1   EnzymeType_A2  18542 non-null  int64  
 2   EnzymeType_B   18542 non-null  int64  
 3   EnzymeType_C   18542 non-null  int64  
 4   EnzymeType_D   18542 non-null  int64  
 5   EnzymeType_E   18542 non-null  int64  
 6   RawMatPercent  18542 non-null  float64
 7   NIRfat         18542 non-null  float64
 8   NIRash         18542 non-null  float64
 9   NIRwater       18542 non-null  float64
 10  TT08           18542 non-null  float64
 11  TT20           18542 non-null  float64
 12  TT12           18542 non-null  float64
 13  Collagen       31 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


In [15]:
collagen_data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10507 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10507 non-null  int64  
 1   EnzymeType_A2  10507 non-null  int64  
 2   EnzymeType_B   10507 non-null  int64  
 3   EnzymeType_C   10507 non-null  int64  
 4   EnzymeType_D   10507 non-null  int64  
 5   EnzymeType_E   10507 non-null  int64  
 6   RawMatPercent  10507 non-null  float64
 7   NIRfat         10507 non-null  float64
 8   NIRash         10507 non-null  float64
 9   NIRwater       10507 non-null  float64
 10  TT08           10507 non-null  float64
 11  TT20           10507 non-null  float64
 12  TT12           10507 non-null  float64
 13  Collagen       58 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Lagring av resultater

In [16]:
# Kolonner til dataframe for lagring av resultater fra opmtimalisering
resultater_kolonner = ["Treningsandel",
                       "RMSE test",
                       "R2 test",
                       "MAE test",
                       "MAPE test",
                       "RMSE trening",
                       "R2 trening",
                       "MAE trening",
                       "MAPE trening"]

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [17]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Definere andel av treningsett som skal benyttes
trenings_andel = 1.0


# Deler datasettet i trening og testsett
treningssett_markert, enzymtyper_treningssett_markert, testsett,\
emzymtyper_testsett,treningssett_umarkert, enzymtyper_treningssett_umarkert,\
treningssett =trening_testsett_oppdeling_enzym(rå_data,
                                               collagen_data,
                                               test_andel=test_andel)


#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_treningssett_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(emzymtyper_testsett.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_treningssett_umarkert.value_counts())



# Treningssett i forklaringsvariabler og respons
X_test = testsett.iloc[:, :-1]
y_test = testsett.iloc[:, -1]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Trening markert
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    7
C     4
B     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [18]:
# Modelloptimalisering
coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                   enzymtyper_treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt=splitt,
                                                   gjentagelser=gjentagelser,
                                                   n_runder=n_runder,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


# Legger resulater til i datasett
coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-19 20:35:08,340] A new study created in memory with name: no-name-99d1960f-3fbb-4e7d-8cfb-0e8716f1be27
[I 2024-03-19 20:36:55,346] Trial 0 finished with value: 4.368198103959848 and parameters: {'k1': 6, 'k2': 4, 'p1': 2, 'p2': 1, 'max_iters': 13, 'pool_size': 167}. Best is trial 0 with value: 4.368198103959848.
[W 2024-03-19 20:37:19,147] Trial 1 failed with parameters: {'k1': 8, 'k2': 6, 'p1': 1, 'p2': 5, 'max_iters': 12, 'pool_size': 54} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/content/moduler_optimalisering_evaluering_semiveiledet.py", line 276, in objective
    coreg.fit(X_trening_fold_markert_transformert,
  File "/usr/local/lib/python3.10/dist-packages/LAMDA_SSL/Algorithm/Regression/CoReg.py", line 79, in fit
    h_temp.predict(L_X[idx_o].reshape(1, -1))) ** 2
  File "/usr/local/li

KeyboardInterrupt: 

### BHD

In [54]:
# BHD
bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               enzymtyper_treningssett_umarkert,
                                               treningssett,
                                               testsett,
                                               X_test,
                                               y_test,
                                               splitt,
                                               gjentagelser,
                                               n_runder,
                                               umarkert_andel=0.25,
                                               trenings_andel=1.0,
                                               random_seed=random_seed)


# Legger resulater til i datasett
bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-19 18:55:23,508] A new study created in memory with name: no-name-7a5c6e5f-b47c-4913-8b8a-c1be1639a76a
[I 2024-03-19 18:57:39,137] Trial 0 finished with value: 5.33967070823602 and parameters: {'k': 6285, 'alpha': 0.1, 'maks_iterasjoner': 101}. Best is trial 0 with value: 5.33967070823602.
[I 2024-03-19 18:59:42,943] Trial 1 finished with value: 9.31455340380905 and parameters: {'k': 5471, 'alpha': 1, 'maks_iterasjoner': 29}. Best is trial 0 with value: 5.33967070823602.


{'k': 6285, 'alpha': 0.1, 'maks_iterasjoner': 101}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,5.701721,0.064514,4.366527,0.229323,1.209543,0.942265,0.908939,0.039667


## Alternativ 1: Flere treningsandeler

In [37]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   collagen_data,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.5, 0.7]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    7
C     4
B     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [72]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                   enzymtyper_treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt=splitt,
                                                   gjentagelser=gjentagelser,
                                                   n_runder=n_runder,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-19 19:10:16,250] A new study created in memory with name: no-name-d5082504-b0e3-4491-a23f-f400ce3cfb7c


Enzymtyper Fordeling: Treningssett markert andel: 0.5
A2    13
C      8
B      8
D      5
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 19:10:23,691] Trial 0 finished with value: 4.4196957334928495 and parameters: {'k1': 9, 'k2': 4, 'p1': 10, 'p2': 9, 'max_iters': 19, 'pool_size': 14}. Best is trial 0 with value: 4.4196957334928495.
[I 2024-03-19 19:10:27,655] A new study created in memory with name: no-name-0bc4fce8-9453-4093-b10b-035d8a0e8352


Enzymtyper Fordeling: Treningssett markert andel: 0.7
A2    18
C     11
B     11
D      8
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 19:10:31,232] Trial 0 finished with value: 4.9286611096899415 and parameters: {'k1': 5, 'k2': 1, 'p1': 3, 'p2': 5, 'max_iters': 1, 'pool_size': 154}. Best is trial 0 with value: 4.9286611096899415.
[I 2024-03-19 19:10:32,613] A new study created in memory with name: no-name-ee9b2a82-2957-4996-bcb2-901959c8fd9d


Enzymtyper Fordeling: Treningssett markert andel: 0.9
A2    23
B     14
C     14
D     10
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-19 19:10:39,398] Trial 0 finished with value: 3.9044307959768414 and parameters: {'k1': 6, 'k2': 8, 'p1': 4, 'p2': 8, 'max_iters': 18, 'pool_size': 18}. Best is trial 0 with value: 3.9044307959768414.


{'trenings_andel': [0.5, 0.7, 0.9], 'parametere': [{'k1': 9, 'k2': 4, 'p1': 10, 'p2': 9, 'max_iters': 19, 'pool_size': 14}, {'k1': 5, 'k2': 1, 'p1': 3, 'p2': 5, 'max_iters': 1, 'pool_size': 154}, {'k1': 6, 'k2': 8, 'p1': 4, 'p2': 8, 'max_iters': 18, 'pool_size': 18}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.5,3.808039,0.582720,2.817819,0.152346,2.697001,0.666389,2.147705,0.089039
1,0.7,4.614026,0.387388,3.850733,0.201283,1.555246,0.895427,1.250396,0.057692
2,0.9,4.171275,0.499317,3.346993,0.178940,2.965527,0.651220,2.376538,0.112246


### BHD

In [77]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-19 20:10:25,479] A new study created in memory with name: no-name-7c6a7a85-f538-40f0-9cc2-71a63dd70649


Enzymtyper Fordeling: Treningssett markert andel: 0.5
A2    13
C      8
B      8
D      5
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 20:11:25,482] Trial 0 finished with value: 5.463257439018376 and parameters: {'k': 6520, 'alpha': 0.01, 'maks_iterasjoner': 59}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:12:20,756] Trial 1 finished with value: 9.440781297034448 and parameters: {'k': 6420, 'alpha': 1, 'maks_iterasjoner': 21}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:13:09,686] Trial 2 finished with value: 13.284018048511529 and parameters: {'k': 3291, 'alpha': 10, 'maks_iterasjoner': 41}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:14:03,260] Trial 3 finished with value: 9.38236450932272 and parameters: {'k': 5615, 'alpha': 1, 'maks_iterasjoner': 67}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:14:24,211] Trial 4 finished with value: 13.283227581124137 and parameters: {'k': 608, 'alpha': 10, 'maks_iterasjoner': 61}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:15:20,603] Trial 5 finished with value: 9.4

Enzymtyper Fordeling: Treningssett markert andel: 0.7
A2    18
C     11
B     11
D      8
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 20:21:47,669] Trial 0 finished with value: 5.170170181366991 and parameters: {'k': 6810, 'alpha': 0.1, 'maks_iterasjoner': 124}. Best is trial 0 with value: 5.170170181366991.
[I 2024-03-19 20:22:46,339] Trial 1 finished with value: 5.170124841396653 and parameters: {'k': 2400, 'alpha': 0.1, 'maks_iterasjoner': 175}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:23:47,677] Trial 2 finished with value: 5.1702718989745815 and parameters: {'k': 6635, 'alpha': 0.1, 'maks_iterasjoner': 75}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:24:27,375] Trial 3 finished with value: 9.511713575788109 and parameters: {'k': 1496, 'alpha': 1, 'maks_iterasjoner': 142}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:24:57,188] Trial 4 finished with value: 25.524638276899225 and parameters: {'k': 1557, 'alpha': 10, 'maks_iterasjoner': 8}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:25:52,165] Trial 5 finished with valu

{'trenings_andel': [0.5, 0.7], 'parametere': [{'k': 2851, 'alpha': 0.01, 'maks_iterasjoner': 177}, {'k': 931, 'alpha': 0.01, 'maks_iterasjoner': 198}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,6.082883,-0.064742,4.513696,0.247507,0.126221,0.999269,0.093372,0.003768
1,1.0,6.111859,-0.074909,4.537162,0.249573,0.132864,0.999237,0.100209,0.004191


## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [17]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 collagen_data,
                                 test_andel=test_andel,
                                 trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_trening_markert = enzymfordeling(rå_data,
                                       trening_markert)
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)

print("Enzymtyper Fordeling: Trening markert")
print(enzym_trening_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzym_trening_umarkert.value_counts())


# umarkert andel til BHD

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Trening markert
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    5
B     4
C     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [19]:
# CoReg
coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                 trening_markert,
                                                 trening_umarkert,
                                                 trening,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 n_runder,
                                                 umarkert_andel=False,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


# Legger resulater til i datasett
coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 12:26:13,581] A new study created in memory with name: no-name-d8641b43-6f32-45dc-8797-45814b88cd11
[I 2024-03-20 12:26:22,443] Trial 0 finished with value: 4.946081362212995 and parameters: {'k1': 1, 'k2': 5, 'p1': 1, 'p2': 1, 'max_iters': 4, 'pool_size': 117}. Best is trial 0 with value: 4.946081362212995.
[I 2024-03-20 12:26:50,295] Trial 1 finished with value: 4.660326319079136 and parameters: {'k1': 3, 'k2': 9, 'p1': 4, 'p2': 6, 'max_iters': 13, 'pool_size': 75}. Best is trial 1 with value: 4.660326319079136.


{'k1': 3, 'k2': 9, 'p1': 4, 'p2': 6, 'max_iters': 13, 'pool_size': 75}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.108279,0.285123,3.434167,0.163272,2.591312,0.760326,2.114892,0.101586


### BHD

In [21]:
# BHD
bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             test,
                                             X_test,
                                             y_test,
                                             splitt,
                                             n_runder,
                                             umarkert_andel=umarkert_andel,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


# Legger resulater til i datasett
bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

[I 2024-03-20 12:28:05,168] A new study created in memory with name: no-name-b1836a9a-da34-450b-8eb1-f40a69d0b3bd
[I 2024-03-20 12:29:19,242] Trial 0 finished with value: 6.394147016986514 and parameters: {'k': 3602, 'alpha': 0.01, 'maks_iterasjoner': 153}. Best is trial 0 with value: 6.394147016986514.
[I 2024-03-20 12:30:30,311] Trial 1 finished with value: 15.263486650709279 and parameters: {'k': 5339, 'alpha': 10, 'maks_iterasjoner': 124}. Best is trial 0 with value: 6.394147016986514.


{'k': 3602, 'alpha': 0.01, 'maks_iterasjoner': 153}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.95906,-0.041623,4.439962,0.21455,0.173033,0.998931,0.141838,0.006167


## Alternativ 2: Flere treningssandeler

In [25]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data,
                                                          test_andel=test_andel,
                                                          trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert andel:", trenings_andel)
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
A2    5
B     4
C     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert andel: 1.0
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [30]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 12:41:47,282] A new study created in memory with name: no-name-19a4510f-938f-4f4e-aa49-dacd2ed75bd4


Enzymtyper Fordeling: Treningssett markert andel: 0.8
A2    25
C     13
B     12
D      9
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 12:42:13,724] Trial 0 finished with value: 4.923731907132404 and parameters: {'k1': 2, 'k2': 1, 'p1': 9, 'p2': 7, 'max_iters': 13, 'pool_size': 160}. Best is trial 0 with value: 4.923731907132404.
[I 2024-03-20 12:42:17,103] Trial 1 finished with value: 4.221967868610392 and parameters: {'k1': 10, 'k2': 8, 'p1': 6, 'p2': 10, 'max_iters': 1, 'pool_size': 52}. Best is trial 1 with value: 4.221967868610392.
[I 2024-03-20 12:42:18,444] A new study created in memory with name: no-name-1c979579-ccfa-4deb-913f-7dc2f4f9c9a9


Enzymtyper Fordeling: Treningssett markert andel: 0.9
A2    28
B     15
C     15
D     10
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 12:42:24,176] Trial 0 finished with value: 4.707078454396344 and parameters: {'k1': 6, 'k2': 6, 'p1': 3, 'p2': 9, 'max_iters': 1, 'pool_size': 192}. Best is trial 0 with value: 4.707078454396344.
[I 2024-03-20 12:42:38,551] Trial 1 finished with value: 4.689424296311372 and parameters: {'k1': 2, 'k2': 3, 'p1': 6, 'p2': 8, 'max_iters': 10, 'pool_size': 89}. Best is trial 1 with value: 4.689424296311372.
[I 2024-03-20 12:42:46,371] A new study created in memory with name: no-name-ec98d4e7-a90f-4f14-a6d2-31555c90387d


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 12:42:47,737] Trial 0 finished with value: 4.957281474697166 and parameters: {'k1': 3, 'k2': 2, 'p1': 3, 'p2': 8, 'max_iters': 1, 'pool_size': 63}. Best is trial 0 with value: 4.957281474697166.
[I 2024-03-20 12:43:03,644] Trial 1 finished with value: 4.481056261481355 and parameters: {'k1': 8, 'k2': 3, 'p1': 7, 'p2': 7, 'max_iters': 3, 'pool_size': 192}. Best is trial 1 with value: 4.481056261481355.


{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'k1': 10, 'k2': 8, 'p1': 6, 'p2': 10, 'max_iters': 1, 'pool_size': 52}, {'k1': 2, 'k2': 3, 'p1': 6, 'p2': 8, 'max_iters': 10, 'pool_size': 89}, {'k1': 8, 'k2': 3, 'p1': 7, 'p2': 7, 'max_iters': 3, 'pool_size': 192}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,4.132425,0.276694,3.535668,0.172233,3.574014,0.541544,2.896961,0.141547
1,0.9,3.856773,0.369972,3.371322,0.154245,1.890206,0.874738,1.472411,0.068334
2,1.0,4.232173,0.241355,3.579433,0.171533,2.756446,0.728806,2.205081,0.108209


### BHD

In [33]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 0.8
A2    25
C     13
B     12
D      9
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 12:44:42,965] A new study created in memory with name: no-name-7c4cb253-04ff-4e06-bf6a-da410603ee39
[I 2024-03-20 12:45:31,128] Trial 0 finished with value: 10.161473580304552 and parameters: {'k': 5847, 'alpha': 1, 'maks_iterasjoner': 12}. Best is trial 0 with value: 10.161473580304552.
[I 2024-03-20 12:46:33,737] Trial 1 finished with value: 5.810216755048612 and parameters: {'k': 4585, 'alpha': 0.1, 'maks_iterasjoner': 200}. Best is trial 1 with value: 5.810216755048612.


Enzymtyper Fordeling: Treningssett markert andel: 0.9
A2    28
B     15
C     15
D     10
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 12:47:05,276] A new study created in memory with name: no-name-c8ecbda7-25e3-4945-98a8-24ce9c2a899b
[I 2024-03-20 12:47:49,278] Trial 0 finished with value: 16.099080236392126 and parameters: {'k': 2898, 'alpha': 10, 'maks_iterasjoner': 63}. Best is trial 0 with value: 16.099080236392126.
[I 2024-03-20 12:48:24,282] Trial 1 finished with value: 6.518597153459472 and parameters: {'k': 2295, 'alpha': 0.1, 'maks_iterasjoner': 9}. Best is trial 1 with value: 6.518597153459472.


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 12:48:43,097] A new study created in memory with name: no-name-e085a81a-c863-470b-b4fb-8989563665e8
[I 2024-03-20 12:49:30,240] Trial 0 finished with value: 15.263566609872452 and parameters: {'k': 4016, 'alpha': 10, 'maks_iterasjoner': 98}. Best is trial 0 with value: 15.263566609872452.
[I 2024-03-20 12:50:27,444] Trial 1 finished with value: 6.394146141199996 and parameters: {'k': 7080, 'alpha': 0.01, 'maks_iterasjoner': 66}. Best is trial 1 with value: 6.394146141199996.


{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'k': 4585, 'alpha': 0.1, 'maks_iterasjoner': 200}, {'k': 2295, 'alpha': 0.1, 'maks_iterasjoner': 9}, {'k': 7080, 'alpha': 0.01, 'maks_iterasjoner': 66}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,4.787196,0.029324,4.182833,0.194204,1.548111,0.913982,1.283088,0.058816
1,0.9,4.857908,0.000436,4.339294,0.201433,1.641226,0.905564,1.346037,0.059098
2,1.0,4.959059,-0.041622,4.439961,0.214550,0.173041,0.998931,0.141845,0.006167


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [36]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   collagen_data_design,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.5, 0.7]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
C     14
B     14
D     11
A2     5
A1     2
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
B     4
C     4
D     3
A2    1
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
D     1972
A1    1842
B     1775
C     1759
E     1706
A2    1395
Name: EnzymeCode, dtype: int64


### CoReg

In [38]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                    enzymtyper_treningssett_markert,
                                                    treningssett_umarkert,
                                                    treningssett,
                                                    X_test,
                                                    y_test,
                                                    splitt=splitt,
                                                    gjentagelser=gjentagelser,
                                                    n_runder=n_runder,
                                                    trenings_andel=trenings_andel,
                                                    random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-20 12:55:03,124] A new study created in memory with name: no-name-fbb7da63-6dc8-4f2a-be1a-6694119ab7c3


Enzymtyper Fordeling: Treningssett markert andel: 0.5
A2    13
C      8
B      8
D      5
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-20 12:55:32,593] Trial 0 finished with value: 4.786764921767511 and parameters: {'k1': 1, 'k2': 9, 'p1': 5, 'p2': 7, 'max_iters': 7, 'pool_size': 139}. Best is trial 0 with value: 4.786764921767511.
[I 2024-03-20 12:55:38,178] Trial 1 finished with value: 4.422368146901587 and parameters: {'k1': 3, 'k2': 8, 'p1': 9, 'p2': 8, 'max_iters': 8, 'pool_size': 21}. Best is trial 1 with value: 4.422368146901587.
[I 2024-03-20 12:56:34,343] Trial 2 finished with value: 4.539371432020677 and parameters: {'k1': 7, 'k2': 10, 'p1': 3, 'p2': 3, 'max_iters': 8, 'pool_size': 159}. Best is trial 1 with value: 4.422368146901587.
[I 2024-03-20 12:57:33,677] Trial 3 finished with value: 4.640516865248354 and parameters: {'k1': 5, 'k2': 7, 'p1': 9, 'p2': 2, 'max_iters': 12, 'pool_size': 170}. Best is trial 1 with value: 4.422368146901587.
[I 2024-03-20 12:57:50,291] Trial 4 finished with value: 4.414757390538573 and parameters: {'k1': 3, 'k2': 9, 'p1': 8, 'p2': 4, 'max_iters': 6, 'pool_size': 82

Enzymtyper Fordeling: Treningssett markert andel: 0.7
A2    18
C     11
B     11
D      8
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:01:39,061] Trial 0 finished with value: 4.432978041874652 and parameters: {'k1': 8, 'k2': 4, 'p1': 8, 'p2': 10, 'max_iters': 20, 'pool_size': 150}. Best is trial 0 with value: 4.432978041874652.
[I 2024-03-20 13:02:12,356] Trial 1 finished with value: 4.607160092251355 and parameters: {'k1': 5, 'k2': 4, 'p1': 3, 'p2': 5, 'max_iters': 16, 'pool_size': 89}. Best is trial 0 with value: 4.432978041874652.
[I 2024-03-20 13:02:39,124] Trial 2 finished with value: 4.3547033961342905 and parameters: {'k1': 5, 'k2': 5, 'p1': 3, 'p2': 10, 'max_iters': 6, 'pool_size': 175}. Best is trial 2 with value: 4.3547033961342905.
[I 2024-03-20 13:03:22,144] Trial 3 finished with value: 4.4672130187373185 and parameters: {'k1': 3, 'k2': 9, 'p1': 7, 'p2': 10, 'max_iters': 12, 'pool_size': 124}. Best is trial 2 with value: 4.3547033961342905.
[I 2024-03-20 13:03:23,844] Trial 4 finished with value: 4.880847360445666 and parameters: {'k1': 6, 'k2': 1, 'p1': 1, 'p2': 6, 'max_iters': 1, 'pool_s

{'trenings_andel': [0.5, 0.7], 'parametere': [{'k1': 4, 'k2': 7, 'p1': 6, 'p2': 10, 'max_iters': 17, 'pool_size': 52}, {'k1': 7, 'k2': 10, 'p1': 5, 'p2': 2, 'max_iters': 3, 'pool_size': 156}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.5,3.783839,0.588006,2.831157,0.153288,2.598100,0.690407,1.998493,0.083649
1,0.7,4.466013,0.426062,3.544012,0.192225,3.312243,0.525685,2.723966,0.124158


### BHD

In [42]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

Enzymtyper Fordeling: Treningssett markert andel: 0.5
A2    13
C      8
B      8
D      5
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:11:07,185] A new study created in memory with name: no-name-5c9c1f0c-92af-4ebc-a67e-0d7d46f542f7
[I 2024-03-20 13:12:25,333] Trial 0 finished with value: 5.463256754560618 and parameters: {'k': 6122, 'alpha': 0.01, 'maks_iterasjoner': 146}. Best is trial 0 with value: 5.463256754560618.
[I 2024-03-20 13:13:30,268] Trial 1 finished with value: 5.639202638176618 and parameters: {'k': 3213, 'alpha': 0.1, 'maks_iterasjoner': 179}. Best is trial 0 with value: 5.463256754560618.
[I 2024-03-20 13:14:43,922] Trial 2 finished with value: 5.639196597479146 and parameters: {'k': 6037, 'alpha': 0.1, 'maks_iterasjoner': 194}. Best is trial 0 with value: 5.463256754560618.
[I 2024-03-20 13:15:31,385] Trial 3 finished with value: 13.287676501053028 and parameters: {'k': 4939, 'alpha': 10, 'maks_iterasjoner': 15}. Best is trial 0 with value: 5.463256754560618.
[I 2024-03-20 13:15:56,648] Trial 4 finished with value: 13.282528272173943 and parameters: {'k': 647, 'alpha': 10, 'maks_iter

Enzymtyper Fordeling: Treningssett markert andel: 0.7
A2    18
C     11
B     11
D      8
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:21:34,700] Trial 0 finished with value: 5.170509900949892 and parameters: {'k': 6680, 'alpha': 0.1, 'maks_iterasjoner': 39}. Best is trial 0 with value: 5.170509900949892.
[I 2024-03-20 13:22:00,558] Trial 1 finished with value: 4.962662119002916 and parameters: {'k': 926, 'alpha': 0.01, 'maks_iterasjoner': 75}. Best is trial 1 with value: 4.962662119002916.
[I 2024-03-20 13:22:56,972] Trial 2 finished with value: 5.170215578514506 and parameters: {'k': 4746, 'alpha': 0.1, 'maks_iterasjoner': 96}. Best is trial 1 with value: 4.962662119002916.
[I 2024-03-20 13:23:39,630] Trial 3 finished with value: 5.172178111437118 and parameters: {'k': 3081, 'alpha': 0.1, 'maks_iterasjoner': 9}. Best is trial 1 with value: 4.962662119002916.
[I 2024-03-20 13:24:45,269] Trial 4 finished with value: 13.681427468131702 and parameters: {'k': 6292, 'alpha': 10, 'maks_iterasjoner': 129}. Best is trial 1 with value: 4.962662119002916.
[I 2024-03-20 13:25:05,424] Trial 5 finished with value

{'trenings_andel': [0.5, 0.7], 'parametere': [{'k': 6122, 'alpha': 0.01, 'maks_iterasjoner': 146}, {'k': 926, 'alpha': 0.01, 'maks_iterasjoner': 75}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.5,6.082883,-0.064742,4.513696,0.247507,0.126222,0.999269,0.093372,0.003768
1,0.7,6.111857,-0.074909,4.537159,0.249573,0.132871,0.999237,0.100214,0.004191


## Alternativ 2: Flere treningssandeler

In [43]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data,
                                                          test_andel=test_andel,
                                                          trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert andel:", trenings_andel)
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
A2    5
B     4
C     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert andel: 0.7
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [44]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 13:29:31,198] A new study created in memory with name: no-name-8c151796-8360-4f99-b570-03c8e9e96a86


Enzymtyper Fordeling: Treningssett markert andel: 0.8
A2    25
C     13
B     12
D      9
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:30:09,472] Trial 0 finished with value: 4.063153568112989 and parameters: {'k1': 10, 'k2': 7, 'p1': 9, 'p2': 5, 'max_iters': 5, 'pool_size': 194}. Best is trial 0 with value: 4.063153568112989.
[I 2024-03-20 13:30:38,974] Trial 1 finished with value: 4.388776930191918 and parameters: {'k1': 9, 'k2': 10, 'p1': 2, 'p2': 9, 'max_iters': 18, 'pool_size': 37}. Best is trial 0 with value: 4.063153568112989.
[I 2024-03-20 13:30:58,067] A new study created in memory with name: no-name-31df0468-59dd-4205-bfec-ff37c30527cf


Enzymtyper Fordeling: Treningssett markert andel: 0.9
A2    28
B     15
C     15
D     10
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:31:02,971] Trial 0 finished with value: 4.843287384741166 and parameters: {'k1': 6, 'k2': 1, 'p1': 9, 'p2': 3, 'max_iters': 1, 'pool_size': 185}. Best is trial 0 with value: 4.843287384741166.
[I 2024-03-20 13:31:04,838] Trial 1 finished with value: 4.620773202274441 and parameters: {'k1': 7, 'k2': 5, 'p1': 7, 'p2': 9, 'max_iters': 5, 'pool_size': 10}. Best is trial 1 with value: 4.620773202274441.
[I 2024-03-20 13:31:05,648] A new study created in memory with name: no-name-fd7e6bf1-30af-4645-b923-9a8242e78f69


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:31:08,469] Trial 0 finished with value: 4.616445298172554 and parameters: {'k1': 10, 'k2': 10, 'p1': 6, 'p2': 9, 'max_iters': 1, 'pool_size': 57}. Best is trial 0 with value: 4.616445298172554.
[I 2024-03-20 13:32:20,167] Trial 1 finished with value: 4.639771920964553 and parameters: {'k1': 8, 'k2': 7, 'p1': 3, 'p2': 1, 'max_iters': 14, 'pool_size': 133}. Best is trial 0 with value: 4.616445298172554.


{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'k1': 10, 'k2': 7, 'p1': 9, 'p2': 5, 'max_iters': 5, 'pool_size': 194}, {'k1': 7, 'k2': 5, 'p1': 7, 'p2': 9, 'max_iters': 5, 'pool_size': 10}, {'k1': 10, 'k2': 10, 'p1': 6, 'p2': 9, 'max_iters': 1, 'pool_size': 57}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,4.149570,0.270680,3.519524,0.170743,3.421576,0.579818,2.707291,0.132383
1,0.9,4.318199,0.210200,3.540343,0.173347,3.111710,0.660533,2.470560,0.122232
2,1.0,4.261632,0.230757,3.637812,0.176493,3.593666,0.539047,2.880736,0.141517


### BHD

In [45]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 0.8
A2    25
C     13
B     12
D      9
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:32:37,295] A new study created in memory with name: no-name-2beaf36c-b2cc-439b-b115-d077d0a3d2f2
[I 2024-03-20 13:32:51,981] Trial 0 finished with value: 1.6708791857887644e+112 and parameters: {'k': 58, 'alpha': 0.01, 'maks_iterasjoner': 37}. Best is trial 0 with value: 1.6708791857887644e+112.
[I 2024-03-20 13:33:49,489] Trial 1 finished with value: 10.003296514408223 and parameters: {'k': 6508, 'alpha': 1, 'maks_iterasjoner': 62}. Best is trial 1 with value: 10.003296514408223.


Enzymtyper Fordeling: Treningssett markert andel: 0.9
A2    28
B     15
C     15
D     10
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:34:18,933] A new study created in memory with name: no-name-306f1396-f778-4465-be46-30c714cf4e8f
[I 2024-03-20 13:34:54,310] Trial 0 finished with value: 16.0989779015189 and parameters: {'k': 1739, 'alpha': 10, 'maks_iterasjoner': 78}. Best is trial 0 with value: 16.0989779015189.
[I 2024-03-20 13:35:59,322] Trial 1 finished with value: 6.517880087572608 and parameters: {'k': 4930, 'alpha': 0.1, 'maks_iterasjoner': 199}. Best is trial 1 with value: 6.517880087572608.


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:36:33,280] A new study created in memory with name: no-name-b39d40df-f535-454f-85c1-f4be94ab627b
[I 2024-03-20 13:37:32,067] Trial 0 finished with value: 10.400717553396776 and parameters: {'k': 4301, 'alpha': 1, 'maks_iterasjoner': 161}. Best is trial 0 with value: 10.400717553396776.
[I 2024-03-20 13:38:19,592] Trial 1 finished with value: 10.449781438702757 and parameters: {'k': 3356, 'alpha': 1, 'maks_iterasjoner': 36}. Best is trial 0 with value: 10.400717553396776.


{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'k': 6508, 'alpha': 1, 'maks_iterasjoner': 62}, {'k': 4930, 'alpha': 0.1, 'maks_iterasjoner': 199}, {'k': 4301, 'alpha': 1, 'maks_iterasjoner': 161}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,9.100632,-2.507962,7.610590,0.351002,10.182009,-2.720934,8.282352,0.380483
1,0.9,4.857229,0.000716,4.339741,0.201496,1.632734,0.906539,1.338769,0.058782
2,1.0,9.042452,-2.463253,7.571808,0.350356,10.936642,-3.269230,8.839193,0.385613


# NIR målinger ekskludert

In [48]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
collagen_data_uten_NIR = collagen_data.drop(columns=NIR_kolonner)
collagen_data_design_uten_NIR = collagen_data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Flere treningsandeler

In [50]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   collagen_data_uten_NIR,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    7
C     4
B     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [51]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                     enzymtyper_treningssett_markert,
                                                     treningssett_umarkert,
                                                     treningssett,
                                                     X_test,
                                                     y_test,
                                                     splitt=splitt,
                                                     gjentagelser=gjentagelser,
                                                     n_runder=n_runder,
                                                     trenings_andel=trenings_andel,
                                                     random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-20 13:41:56,278] A new study created in memory with name: no-name-43635a3c-b79b-4d96-8b8d-bc029bfc873f


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:42:09,724] Trial 0 finished with value: 5.050093033762192 and parameters: {'k1': 8, 'k2': 9, 'p1': 4, 'p2': 10, 'max_iters': 20, 'pool_size': 12}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:02,888] Trial 1 finished with value: 5.073281057754931 and parameters: {'k1': 5, 'k2': 10, 'p1': 4, 'p2': 7, 'max_iters': 17, 'pool_size': 65}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:19,090] Trial 2 finished with value: 5.399002928636507 and parameters: {'k1': 10, 'k2': 2, 'p1': 10, 'p2': 3, 'max_iters': 18, 'pool_size': 27}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:38,410] Trial 3 finished with value: 5.260300386083797 and parameters: {'k1': 10, 'k2': 2, 'p1': 6, 'p2': 6, 'max_iters': 2, 'pool_size': 148}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:41,041] Trial 4 finished with value: 4.916827370608138 and parameters: {'k1': 9, 'k2': 7, 'p1': 5, 'p2': 3, 'max_iters': 5, 'pool_size'

{'trenings_andel': [1.0], 'parametere': [{'k1': 6, 'k2': 9, 'p1': 8, 'p2': 9, 'max_iters': 1, 'pool_size': 34}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.942628,0.297023,4.232296,0.222778,3.937273,0.388229,3.213241,0.152856


### BHD

In [52]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-20 13:46:46,443] A new study created in memory with name: no-name-bca6e7c0-187c-4bea-bce9-1a587c77e6b0


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:47:56,537] Trial 0 finished with value: 5.027610544334957 and parameters: {'k': 5051, 'alpha': 0.1, 'maks_iterasjoner': 146}. Best is trial 0 with value: 5.027610544334957.
[I 2024-03-20 13:48:49,062] Trial 1 finished with value: 5.027608547256884 and parameters: {'k': 2715, 'alpha': 0.1, 'maks_iterasjoner': 176}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:49:25,751] Trial 2 finished with value: 6.2877293088414685 and parameters: {'k': 1881, 'alpha': 1, 'maks_iterasjoner': 89}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:50:32,392] Trial 3 finished with value: 6.285797254057214 and parameters: {'k': 5743, 'alpha': 1, 'maks_iterasjoner': 128}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:51:00,931] Trial 4 finished with value: 9.354421732500974 and parameters: {'k': 1092, 'alpha': 10, 'maks_iterasjoner': 96}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:52:00,997] Trial 5 finished with value:

{'trenings_andel': [1.0], 'parametere': [{'k': 751, 'alpha': 0.01, 'maks_iterasjoner': 126}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,5.99464,-0.034074,4.49963,0.241266,0.072208,0.999794,0.059269,0.002705


## Alternativ 2: Flere treningssandeler

In [56]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data_uten_NIR,
                                                          test_andel=test_andel,
                                                          trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert andel:", trenings_andel)
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
A2    5
B     4
C     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert andel: 1.0
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [54]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 13:54:14,192] A new study created in memory with name: no-name-ca9e0276-d8ce-43a3-a29e-868aa4cf440c


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:54:25,918] Trial 0 finished with value: 5.946402474305579 and parameters: {'k1': 10, 'k2': 6, 'p1': 8, 'p2': 3, 'max_iters': 2, 'pool_size': 154}. Best is trial 0 with value: 5.946402474305579.
[I 2024-03-20 13:56:00,212] Trial 1 finished with value: 6.0440976840206515 and parameters: {'k1': 6, 'k2': 4, 'p1': 5, 'p2': 10, 'max_iters': 20, 'pool_size': 186}. Best is trial 0 with value: 5.946402474305579.


{'trenings_andel': [1.0], 'parametere': [{'k1': 10, 'k2': 6, 'p1': 8, 'p2': 3, 'max_iters': 2, 'pool_size': 154}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.687323,0.069403,4.022689,0.196577,3.958656,0.440659,3.245401,0.158872


### BHD

In [55]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:56:08,112] A new study created in memory with name: no-name-6a664a9a-e232-41ee-8994-ea442fbf3316
[I 2024-03-20 13:56:56,654] Trial 0 finished with value: 13.230903344108082 and parameters: {'k': 2315, 'alpha': 10, 'maks_iterasjoner': 178}. Best is trial 0 with value: 13.230903344108082.
[I 2024-03-20 13:57:24,290] Trial 1 finished with value: 8.072900120392212 and parameters: {'k': 372, 'alpha': 1, 'maks_iterasjoner': 200}. Best is trial 1 with value: 8.072900120392212.


{'trenings_andel': [1.0], 'parametere': [{'k': 372, 'alpha': 1, 'maks_iterasjoner': 200}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,6.973406,-1.059689,5.683795,0.255547,8.019235,-1.295342,6.235015,0.266181


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [57]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   collagen_data_design_uten_NIR,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
C     14
B     14
D     11
A2     5
A1     2
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
B     4
C     4
D     3
A2    1
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
D     1972
A1    1842
B     1775
C     1759
E     1706
A2    1395
Name: EnzymeCode, dtype: int64


### CoReg

In [58]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                    enzymtyper_treningssett_markert,
                                                    treningssett_umarkert,
                                                    treningssett,
                                                    X_test,
                                                    y_test,
                                                    splitt=splitt,
                                                    gjentagelser=gjentagelser,
                                                    n_runder=n_runder,
                                                    trenings_andel=trenings_andel,
                                                    random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-20 13:58:16,000] A new study created in memory with name: no-name-3155279c-fd73-46e6-94e4-f71b428be738


Enzymtyper Fordeling: Treningssett markert andel: 1.0
C     14
B     14
D     11
A2     5
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:58:28,682] Trial 0 finished with value: 4.349148479433177 and parameters: {'k1': 7, 'k2': 10, 'p1': 4, 'p2': 4, 'max_iters': 2, 'pool_size': 163}. Best is trial 0 with value: 4.349148479433177.
[I 2024-03-20 13:58:30,026] Trial 1 finished with value: 4.638253779292187 and parameters: {'k1': 2, 'k2': 2, 'p1': 10, 'p2': 7, 'max_iters': 1, 'pool_size': 70}. Best is trial 0 with value: 4.349148479433177.
[I 2024-03-20 13:58:49,914] Trial 2 finished with value: 4.432419371861561 and parameters: {'k1': 9, 'k2': 7, 'p1': 4, 'p2': 1, 'max_iters': 8, 'pool_size': 70}. Best is trial 0 with value: 4.349148479433177.
[I 2024-03-20 13:59:17,923] Trial 3 finished with value: 4.608303729710822 and parameters: {'k1': 1, 'k2': 9, 'p1': 7, 'p2': 2, 'max_iters': 17, 'pool_size': 90}. Best is trial 0 with value: 4.349148479433177.
[I 2024-03-20 13:59:28,677] Trial 4 finished with value: 4.71080268725479 and parameters: {'k1': 1, 'k2': 3, 'p1': 6, 'p2': 2, 'max_iters': 5, 'pool_size': 154}

{'trenings_andel': [1.0], 'parametere': [{'k1': 6, 'k2': 3, 'p1': 1, 'p2': 10, 'max_iters': 5, 'pool_size': 154}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.588689,-0.019148,3.853889,0.189624,3.170894,0.603964,2.57629,0.116297


### BHD

In [59]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-20 14:00:21,938] A new study created in memory with name: no-name-91250566-f45f-4c07-a896-bf4b2b4e11ea


Enzymtyper Fordeling: Treningssett markert andel: 1.0
C     14
B     14
D     11
A2     5
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:00:30,633] Trial 0 finished with value: 7.0477430036266195 and parameters: {'k': 2322, 'alpha': 1, 'maks_iterasjoner': 144}. Best is trial 0 with value: 7.0477430036266195.
[I 2024-03-20 14:00:36,993] Trial 1 finished with value: 9.928171835120885 and parameters: {'k': 1405, 'alpha': 10, 'maks_iterasjoner': 113}. Best is trial 0 with value: 7.0477430036266195.
[I 2024-03-20 14:00:43,501] Trial 2 finished with value: 7.4522799199520735 and parameters: {'k': 2582, 'alpha': 1, 'maks_iterasjoner': 3}. Best is trial 0 with value: 7.0477430036266195.
[I 2024-03-20 14:00:50,428] Trial 3 finished with value: 5.085764895284852 and parameters: {'k': 1909, 'alpha': 0.1, 'maks_iterasjoner': 93}. Best is trial 3 with value: 5.085764895284852.
[I 2024-03-20 14:00:59,400] Trial 4 finished with value: 9.92754531897237 and parameters: {'k': 2198, 'alpha': 10, 'maks_iterasjoner': 147}. Best is trial 3 with value: 5.085764895284852.
[I 2024-03-20 14:01:02,658] Trial 5 finished with value

{'trenings_andel': [1.0], 'parametere': [{'k': 490, 'alpha': 0.01, 'maks_iterasjoner': 21}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.988597,-0.204528,4.089666,0.206132,0.091019,0.999674,0.066162,0.003041


## Alternativ 2: Flere treningssandeler

In [60]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data,
                                                          test_andel=test_andel,
                                                          trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert andel:", trenings_andel)
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
A2    5
B     4
C     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert andel: 1.0
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [61]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 14:01:31,183] A new study created in memory with name: no-name-c70a17e9-3462-4d62-9ffc-39fa43177afc


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:02:14,524] Trial 0 finished with value: 4.744274425811353 and parameters: {'k1': 10, 'k2': 6, 'p1': 5, 'p2': 2, 'max_iters': 8, 'pool_size': 118}. Best is trial 0 with value: 4.744274425811353.
[I 2024-03-20 14:02:26,454] Trial 1 finished with value: 4.608249881173862 and parameters: {'k1': 7, 'k2': 10, 'p1': 5, 'p2': 8, 'max_iters': 8, 'pool_size': 36}. Best is trial 1 with value: 4.608249881173862.


{'trenings_andel': [1.0], 'parametere': [{'k1': 7, 'k2': 10, 'p1': 5, 'p2': 8, 'max_iters': 8, 'pool_size': 36}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.363408,0.193576,3.671959,0.178268,3.321593,0.606201,2.629182,0.129072


### BHD

In [62]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:02:33,802] A new study created in memory with name: no-name-2637f184-92b8-481b-b30b-a576cf014d62
[I 2024-03-20 14:03:06,355] Trial 0 finished with value: 11.875057535216705 and parameters: {'k': 1786, 'alpha': 1, 'maks_iterasjoner': 2}. Best is trial 0 with value: 11.875057535216705.
[I 2024-03-20 14:04:00,607] Trial 1 finished with value: 6.394147053894678 and parameters: {'k': 2593, 'alpha': 0.01, 'maks_iterasjoner': 162}. Best is trial 1 with value: 6.394147053894678.


{'trenings_andel': [1.0], 'parametere': [{'k': 2593, 'alpha': 0.01, 'maks_iterasjoner': 162}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.95906,-0.041623,4.439962,0.21455,0.173033,0.998931,0.141838,0.006167
